In [1]:
from datetime import datetime, date
from pandas import DataFrame as df
from pathlib import Path
import pandas as pd
from os import path
import datetime
import time
import os

pd.options.display.float_format = "{:,.2f}".format

from mthly_hedge_file import HedgeFile

%load_ext autoreload
%autoreload 2

In [2]:
results_fldr = r'C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results'

In [3]:
def get_removed_items(orig_df, true_df):
    """
    Returns a dataframe containing only records that have been removed
    """
    join_cols = ['HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget']
    rename_cols = ['PolicyCount', 'Base_Liab_Ntnl', 'Notional']
    final_cols = ['Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']

    rename_cols_prev = { col: col + '_Prev' for col in rename_cols}
    rename_cols_curr = { col: col + '_Curr' for col in rename_cols}

    prev_df = orig_df.rename(columns=rename_cols_prev, errors="raise")
    curr_df = true_df.rename(columns=rename_cols_curr, errors="raise") # .drop(columns=['Record_Type', 'Co_Code', 'Val_Date', 'Best_Val_Ind'])

    # join_cols = ['Fund_ID', 'Eff_Date', 'Price_Eff_Dt', 'Cap_Rate', 'Cap_Plus_Accel', 'Part_Rate', 'Price_Reset', 'Best_Val_Ind']
    
    maturing_df = pd.merge(prev_df, curr_df, how='outer', on=join_cols, indicator=True)

    # maturing_df = pd.DataFrame(self.prev_results[maturing_df['_merge'] == 'left_only'])
    maturing_df = pd.DataFrame(maturing_df[maturing_df['_merge'] == 'left_only'])
    
    maturing_df['PolicyCount'] = -maturing_df['PolicyCount_Prev']
    maturing_df['Base_Liab_Ntnl'] = -maturing_df['Base_Liab_Ntnl_Prev']
    maturing_df['Notional'] = -maturing_df['Notional_Prev']
    maturing_df['Attrib_Type'] = 'Removed'
    maturing_df = maturing_df[final_cols]

    return maturing_df

    

    

In [4]:
def get_new_items(orig_df, true_df):
    """
    Returns a dataframe containing only records that are new
    """
    join_cols = ['HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget']
    rename_cols = ['PolicyCount', 'Base_Liab_Ntnl', 'Notional']
    final_cols = ['Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']

    rename_cols_prev = { col: col + '_Prev' for col in rename_cols}
    rename_cols_curr = { col: col + '_Curr' for col in rename_cols}

    prev_df = orig_df.rename(columns=rename_cols_prev, errors="raise")
    curr_df = true_df.rename(columns=rename_cols_curr, errors="raise") # .drop(columns=['Record_Type', 'Co_Code', 'Val_Date', 'Best_Val_Ind'])

    new_df = pd.merge(prev_df, curr_df, how='outer', on=join_cols, indicator=True)
    new_df = pd.DataFrame(new_df[new_df['_merge'] == 'right_only'])
    
    new_df['PolicyCount'] = new_df['PolicyCount_Curr']
    new_df['Base_Liab_Ntnl'] = new_df['Base_Liab_Ntnl_Curr']
    new_df['Notional'] = new_df['Notional_Curr']
    new_df['Attrib_Type'] = 'Added'

    new_df = new_df[final_cols]

    return new_df

In [5]:
def get_changed_items(orig_df, true_df):
    """
    Returns a dataframe of changed items that exist both on the orig and true-up files
    """
    join_cols = ['HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget']
    rename_cols = ['PolicyCount', 'Base_Liab_Ntnl', 'Notional']
    final_cols = ['Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']

    rename_cols_prev = { col: col + '_Prev' for col in rename_cols}
    rename_cols_curr = { col: col + '_Curr' for col in rename_cols}

    prev_df = orig_df.rename(columns=rename_cols_prev, errors="raise")
    curr_df = true_df.rename(columns=rename_cols_curr, errors="raise") # .drop(columns=['Record_Type', 'Co_Code', 'Val_Date', 'Best_Val_Ind'])

    chg_df = pd.merge(prev_df, curr_df, how='inner', on=join_cols)
    
    chg_df['PolicyCount'] = chg_df['PolicyCount_Curr'] - chg_df['PolicyCount_Prev']
    chg_df['Base_Liab_Ntnl'] = chg_df['Base_Liab_Ntnl_Curr'] - chg_df['Base_Liab_Ntnl_Prev']
    chg_df['Notional'] = chg_df['Notional_Curr'] - chg_df['Notional_Prev']
    chg_df['Attrib_Type'] = 'Changed'

    chg_df = chg_df[final_cols]

    chg_df = chg_df[chg_df['Base_Liab_Ntnl']!=0]
        
    return chg_df


In [6]:
def create_attrib_df(orig_df, true_df):
        """
        Creates a dataframe from removed items, new items and change in existing items
        """        
        attrib_df = pd.concat([get_removed_items(orig_df, true_df), get_new_items(orig_df, true_df), get_changed_items(orig_df, true_df)]).reset_index(drop=True)
        
        return attrib_df

In [51]:
def process_subdir_hdgfiles(target_dir, directories_to_process=None):
    """
    Iterates over the subdirectories of a given directory and prints their paths.

    Args:
        target_dir: The path to the directory to iterate over.
    """
    
    final_cols = ['HedgeFile_Dt', 'Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']
    df_cols = ['HedgeDate',	'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']


    orig_file = 'iul_liab_summary.csv'
    true_file = 'iul_liab_summary_TRUE_UP.csv'
    results_df = None

    target_dir = Path(target_dir)

    if directories_to_process is None:
        # Set directories to all the valid directories
        # directories_to_process = [x[1] for x  in os.walk(target_dir)]
        # directories_to_process = directories_to_process[0]
        directories_to_process = [f.name for f in target_dir.iterdir() if f.is_dir()]
    
    for dirpath, dirnames, filenames in os.walk(target_dir):
        
        for dirname in dirnames:

            if dirname in directories_to_process:

                # found_orig, found_true = False
                inforce_dt = hedge_dt = None

                subdir_path = os.path.join(dirpath, dirname)

                if os.path.isdir(subdir_path):

                    orig_path = os.path.join(subdir_path, orig_file)
                    true_path = os.path.join(subdir_path, true_file)

                    if os.path.exists(orig_path):
                        # found_orig = True
                        orig_df = pd.read_csv(orig_path)
                        orig_df = orig_df[df_cols]

                        orig_df['HedgeDate'] = pd.to_datetime(orig_df['HedgeDate']).dt.date
                        hedge_dt = orig_df['HedgeDate'].max()
                        
                        # file_name = os.path.basename(orig_path)
                        # hdg_mth = int(file_name[:2])
                        # hdg_yr = int(file_name[3:7])
                        # inforce_dt = date(hdg_yr, hdg_mth, 1)
                        

                    if os.path.exists(true_path):
                        # found_true = True
                        true_df = pd.read_csv(true_path)
                        true_df = true_df[df_cols]
                        true_df['HedgeDate'] = pd.to_datetime(true_df['HedgeDate']).dt.date

                    # Create diffs summary if both files are found
                    # if found_orig and found_true:
                    # NOTE: Copied orig to true-up or vice versa in results dir to make sure both existed in every case (results will be 0 in this case, since no diffs will be found)
                    chgs_df = create_attrib_df(orig_df, true_df)

                    # orig_df['Attrib_Type'] = 'New' if orig_df['HedgeDate']==hedge_dt else 'Added'
                    orig_df['Attrib_Type'] = orig_df.apply(lambda row: 'New' if row['HedgeDate']==hedge_dt else 'Added', axis=1)

                    orig_df['HedgeFile_Dt'] = hedge_dt
                    chgs_df['HedgeFile_Dt'] = hedge_dt

                    orig_df = orig_df[final_cols]
                    chgs_df = chgs_df[final_cols]

                    if results_df is not None:
                        results_df = pd.concat([results_df, orig_df, chgs_df]).reset_index(drop=True)
                    else:
                        results_df = pd.concat([orig_df, chgs_df]).reset_index(drop=True)

    return results_df

In [14]:
test_dirs = ['202405']
test_df = process_subdir_hdgfiles(results_fldr, directories_to_process=test_dirs)
test_df.to_clipboard()

In [50]:
test_fldr = Path(results_fldr)
print(test_fldr)

test_process = [f.name for f in test_fldr.iterdir() if f.is_dir()]
test_process

C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results


['202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312',
 '202401',
 '202402',
 '202403',
 '202404',
 '202405',
 '202406',
 '202407',
 '202408',
 '202409',
 '202410',
 '202411',
 '202412']

In [58]:
full_results_df = process_subdir_hdgfiles(results_fldr)

In [59]:
full_results_df

,HedgeFile_Dt,Attrib_Type,HedgeDate,CompID,Indicator,Bbg_Idx,Fund_Name,Opt_Type,Strike,Cap/Rate,Budget,PolicyCount,Base_Liab_Ntnl,Notional
0,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,20.00,"3,287.51","3,188.88"
1,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,7.00,"5,043.19","4,891.89"
2,2023-07-03,New,2023-07-03,1,NASIDX,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.06,"1,607.00","270,547.46","262,431.04"
3,2023-07-03,New,2023-07-03,1,INDEX,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.07,0.04,373.00,"171,547.68","166,401.25"
4,2023-07-03,New,2023-07-03,1,INDEX,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,1.00,0.10,0.05,90.00,"509,891.40","494,594.66"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2024-12-02,Changed,2024-12-02,26,INXFEE,SPX Index,S&P 500 1 Yr Pt-to-Pt Uncapped w Fee,Call,1.09,10.00,0.03,0.00,7.91,6.01
1784,2024-12-02,Changed,2024-12-02,1,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.06,0.04,5.00,464.70,441.47
1785,2024-12-02,Changed,2024-12-02,1,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.07,0.05,25.00,"8,682.88","8,248.74"
1786,2024-12-02,Changed,2024-12-02,26,INXSPC,SPX Index,S&P 500 1 Yr Specified Rate,Digital,1.00,0.07,0.05,8.00,497.07,377.77


In [61]:
full_results_df.to_clipboard()

In [33]:
# test_sub_dirs = [x[1] for x  in os.walk(results_fldr)]
test_sub_dirs = [x[1] for x in os.walk(results_fldr)]
test_sub_dirs = test_sub_dirs[0]

print(test_sub_dirs)

['202307', '202308', '202309', '202310', '202311', '202312', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408', '202409', '202410', '202411', '202412']


In [28]:
full_results_df

In [ ]:
test_sub_dir = '202405'
test_dir = os.path.join(results_fldr, test_sub_dir)

orig_file = 'iul_liab_summary.csv'
true_file = 'iul_liab_summary_TRUE_UP.csv'

orig_path = os.path.join(test_dir, orig_file)
true_path = os.path.join(test_dir, true_file)

final_cols = ['HedgeFile_Dt', 'Attrib_Type', 'HedgeDate', 'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']
df_cols = ['HedgeDate',	'CompID', 'Indicator', 'Bbg_Idx', 'Fund_Name', 'Opt_Type', 'Strike', 'Cap/Rate', 'Budget', 'PolicyCount', 'Base_Liab_Ntnl', 'Notional']

orig_df = pd.read_csv(orig_path)
orig_df = orig_df[df_cols]

true_df = pd.read_csv(true_path)
true_df = true_df[df_cols]

orig_df['HedgeDate'] = pd.to_datetime(orig_df['HedgeDate']).dt.date
true_df['HedgeDate'] = pd.to_datetime(true_df['HedgeDate']).dt.date
hedge_dt = orig_df['HedgeDate'].max()


In [55]:
test_file = r'G:\MarketRiskMgmt\Pricing Requests\2024-Orion - IUL Hedging\RGA_Process\2_Results\202412\Review Automated VBA\12_2024_HEDGE_ORIG.txt'
hedgefile_colnames = ['CompID', 'PolicyNum', 'Plan', 'IssueDate', 'Indicator', 'Tranx', 'Rev', 'Dept_Desk', 'Entry_Date', 'AsOf_Date', 'Base_Liab_Ntnl', 'Part', 'Cap', 'MGIR_(Cap)', 'Floor', 'Spec_Rate', 'Spread', 'Asset_Charge', 'Multiplier']

test_df = pd.read_csv(test_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python')

C:\Users\S0053071\AppData\Local\Temp\ipykernel_32400\3241378652.py:4: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  test_df = pd.read_csv(test_file, index_col=False, header=None, names=hedgefile_colnames, skipfooter=2, engine='python')


In [57]:
test_df['Base_Liab_Ntnl'].sum()

np.float64(142757763.13)